In [1]:
import json
from IPython.display import display, Image
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import Image as WidgetImage
import csv
import os
from collections import defaultdict
from os import listdir
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
from pattern import comparative
import base64
from os.path import join
from collections import Counter

# create source data

In [27]:
data_path = 'cub_200_2011/att_cls_images/test'
folders = list(os.scandir(data_path))

question_list = json.load(open('cub_questions.json'))
all_name = []
c = []
for folder in folders:
    if folder.name == '.DS_Store':
        continue
    info = json.load(open(join(folder, 'info.json')))
    all_data = [join(data_path, folder.name, i) for i in info['images']]
    data_l = len(all_data)
    question = question_list[info['attribute']]
    tmp = {}
    if data_l < 2:
        continue
    if data_l > 20:
        random.shuffle(all_data)
        tmp = {}
        sub_list = []
        for i in all_data:
            sub_list.append(i)
            if len(sub_list) == 20:
                tmp['question'] = question
                tmp['images'] = sub_list
                tmp['folder'] = folder.name
                tmp['attribute'] = info['attribute']
                all_name.append(tmp)
                c.append(len(sub_list))
                tmp = {}
                sub_list = []
        if len(sub_list) > 1:
            tmp['question'] = question
            tmp['images'] = sub_list
            tmp['folder'] = folder.name
            tmp['attribute'] = info['attribute']
            all_name.append(tmp)
            c.append(len(sub_list))
        elif len(sub_list) > 0 and len(sub_list) < 2:
            all_name[-1]['images'] += sub_list
            c[-1] += 1
    else:
        tmp['question'] = question
        tmp['images'] = all_data
        tmp['folder'] = folder.name
        tmp['attribute'] = info['attribute']
        all_name.append(tmp)
        c.append(len(all_data))

In [28]:
Counter(c)

Counter({20: 451,
         2: 375,
         3: 277,
         4: 214,
         5: 183,
         6: 126,
         7: 105,
         9: 103,
         8: 101,
         12: 78,
         10: 72,
         15: 67,
         11: 65,
         14: 59,
         13: 54,
         18: 52,
         16: 48,
         19: 45,
         17: 37,
         21: 34})

In [29]:
len(all_name)

2546

In [30]:
all_name[0]

{'question': 'Which bird has more red on its belly?',
 'images': ['cub_200_2011/att_cls_images/test/211_41/Scissor_Tailed_Flycatcher_0059_41998.jpg',
  'cub_200_2011/att_cls_images/test/211_41/Scissor_Tailed_Flycatcher_0025_42032.jpg',
  'cub_200_2011/att_cls_images/test/211_41/Scissor_Tailed_Flycatcher_0100_41796.jpg',
  'cub_200_2011/att_cls_images/test/211_41/Scissor_Tailed_Flycatcher_0121_41843.jpg',
  'cub_200_2011/att_cls_images/test/211_41/Scissor_Tailed_Flycatcher_0082_41852.jpg'],
 'folder': '211_41',
 'attribute': 'has_belly_color::red'}

In [31]:
with open('CUB_source.json', 'w') as fout:
    json.dump(all_name, fout)

# Label starts

In [36]:
def load_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)
    
class label_tool():
    def __init__(self, json_name, start, end):
        self.current_index = start
        self.data = load_data(json_name) 
        self.results = []
        self.end = end
        self.start = start
        # Create widgets
        self.left_text = widgets.Text(description="Left:", layout={'width': '80%'})
        self.right_text = widgets.Text(description="Right:", layout={'width': '80%'})
        self.reset_button = widgets.Button(description="Reset")
        self.save_next_button = widgets.Button(description="Save and Next")
        self.next_list_button = widgets.Button(description="Next List")
        self.question_text = widgets.Text(description="Question:",layout={'width': '80%'})
        #self.type_buttons = widgets.RadioButtons(options=['Size', 'Color', 'Texture', 'Shape', 'Pattern', 'State', 'None'], value='None', description='Type:')
        self.answer_buttons = widgets.RadioButtons(options=['Left', 'Right', 'None'], value='None', description='Answer:')
        self.back_buttons = widgets.Button(description="Back")
        self.image_widgets = []
        
    def create_image_widget(self, path):
        """Create a widget for each image with radio buttons and display the first 10 letters of the path."""
        if os.path.exists(path):
            image_file = open(path, 'rb').read()
        else:
            # Provide a placeholder if the image isn't found
            image_file = widgets.Label(value="Image not found").value

        image = widgets.Image(value=image_file, format='jpg')
        radio_buttons = widgets.RadioButtons(
            options=['Left', 'Right', 'None'],
            value='None',  # Default value set to 'None'
            description='',
            disabled=False,
            layout={'width': 'max-content'}
        )
        
        name = path.split('/')[-1]
        path_label = widgets.Label(value=name[:20])  # Display the first 10 letters of the path

        radio_buttons.observe(lambda change, p=path, rb=radio_buttons: self.on_radio_change(change, p, rb), names='value')

        box = widgets.VBox([image, radio_buttons, path_label])
        return box
    
    def display_images(self):
        """Display all images for a given list index."""
        clear_output()
        self.image_widgets[self.current_index] = []
        
        
        for path in self.data[self.current_index]['images']:
            widget = self.create_image_widget(path)
            self.image_widgets[self.current_index].append(widget)

        grid = widgets.GridBox(self.image_widgets[self.current_index], layout=widgets.Layout(grid_template_columns="repeat(3, 330px)"))
        display(grid)
        print(self.data[self.current_index]['attribute'])
        display(widgets.VBox([self.question_text]))
        display(self.left_text, self.right_text, self.answer_buttons, self.reset_button, self.save_next_button, self.next_list_button, self.back_buttons)
        
    
    
    def on_radio_change(self, change, path, current_radio):
        """Handle radio button state changes and ensure exclusivity for Left and Right selections."""
        name = path.split('/')[-1]
        if change['new'] in ['Left', 'Right']:
            # Reset other selections
            for widget_list in self.image_widgets[self.current_index]:
                if widget_list.children[1] is not current_radio and widget_list.children[1].value == change['new']:
                    widget_list.children[1].value = 'None'
            # Update text boxes
            if change['new'] == 'Left':
                self.left_text.value = name
            elif change['new'] == 'Right':
                self.right_text.value = name
        else:
            # Clear the text boxes if unselected
            if self.left_text.value == name:
                self.left_text.value = ''
            if self.right_text.value == name:
                self.right_text.value = ''
    
    def reset_all(self, button):
        """Reset all text boxes, radio buttons to 'None', and clear answer and type selections."""
        self.left_text.value = ''
        self.right_text.value = ''
#         self.type_buttons.value = 'None'
        self.answer_buttons.value = None  # Deselect any selected answer
        for widget_list in self.image_widgets[self.current_index]:
            widget_list.children[1].value = 'None'

    def save_and_next(self, button):
        """Save paths, type, answer, and update the question."""
        left = self.image_root + '/' + self.left_text.value
        right = self.image_root + '/' + self.right_text.value

        self.results.append({'image_1': left, 'image_2': right, 
#                              'type': self.type_buttons.value, 
                             'answer': self.answer_buttons.value, 
                             'question': self.question_text.value})

        print(self.results[-1])
        self.reset_all(None)

    def next_list(self, button):
        """Move to the next list of images and update question text."""
        self.current_index += 1
        if  self.current_index < self.end:
            self.question_text.value = self.data[self.current_index]['question']
            self.image_root = '/'.join(self.data[self.current_index]['images'][0].split('/')[:3])  # Update the question box with the new question
            self.display_images()
        else:
            print("No more lists available.")
    
    def back_function(self, button):
        self.results.pop()
        self.reset_all(None)
        print('remove last pair')
    
    
    def save_updated_data(self):
        with open(f'CUB_{self.start}_{self.end}.json', 'w') as file:
            json.dump(self.results, file)
    
    def begin(self):
        # Initialize the image_widgets list for each list of images
        for _ in range(len(self.data)):
            self.image_widgets.append([])
        self.display_images()
        self.question_text.value = self.data[self.current_index]['question']
        self.image_root = '/'.join(self.data[self.current_index]['images'][0].split('/')[:3])
        # Link buttons to functions
        self.reset_button.on_click(self.reset_all)
        self.save_next_button.on_click(self.save_and_next)
        self.next_list_button.on_click(self.next_list)
        self.back_buttons.on_click(self.back_function)

In [33]:
labeler = label_tool('CUB_source.json',100,105)

In [34]:
labeler.begin()

GridBox(children=(VBox(children=(Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\…

has_tail_shape::squared_tail


Text(value='', description='Left:', layout=Layout(width='80%'))

Text(value='', description='Right:', layout=Layout(width='80%'))

RadioButtons(description='Answer:', options=('Left', 'Right', 'None'), value=None)

Button(description='Reset', style=ButtonStyle())

Button(description='Save and Next', style=ButtonStyle())

Button(description='Next List', style=ButtonStyle())

Button(description='Back', style=ButtonStyle())

In [35]:
labeler.results

[{'image_1': 'cub_200_2011/att_cls_images/test/Sayornis_0130_98678.jpg',
  'image_2': 'cub_200_2011/att_cls_images/test/Sayornis_0094_98512.jpg',
  'answer': 'Right',
  'question': 'Which bird has more brown on its crown?'}]

In [55]:
labeler.save_updated_data()